In [3]:
%pip install -U -q transformers faiss-cpu torch sentence-transformers pillow opencv-python

In [4]:
import torch
import faiss
import cv2
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer

In [5]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
image_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

text_model = SentenceTransformer("all-MiniLM-L6-v2")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
dataset_texts = ["A cat sitting on a table", "A dog playing in the park", "A red sports car", "A bowl of fresh fruit"]
dataset_images = ["cat.jpg", "dog.jpg", "car.jpg", "fruit.jpg"]

text_embeddings = text_model.encode(dataset_texts, convert_to_tensor=True)

image_embeddings = []
for img_path in dataset_images:
    image = Image.open(img_path).convert("RGB")
    inputs = processor(image, return_tensors="pt")
    img_emb = image_model.generate(**inputs)
    image_embeddings.append(img_emb)

image_embeddings = torch.cat(image_embeddings)

In [ ]:
data_embeddings = torch.cat((text_embeddings, image_embeddings)).detach().numpy()

index = faiss.IndexFlatL2(data_embeddings.shape[1])
index.add(data_embeddings)

In [ ]:
query_text = "A cute kitten"
query_embedding = text_model.encode([query_text], convert_to_tensor=True).detach().numpy()

distances, indices = index.search(query_embedding, k=3)

print("Top 3 nearest MultiModal results:", indices)